item-based sander en hugo

In [7]:
import json
from pandas import Series, Timestamp
import pandas as pd
from pandas import DataFrame
from numpy import nan
import sklearn.metrics.pairwise as pw


In [8]:
def load_jsons(data_path, file):
    """ helper function to load '.json' files (they're not proper jsons) """
    file_path = data_path + file
    with open(file_path) as jsons:
        lines = [json.loads(json_line) for json_line in jsons]
    return pd.DataFrame(lines)

In [54]:
data_path = '../data/oregon/'
review_file = 'review.json'
reviews = load_jsons(data_path, review_file)

display(reviews)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,MjA_onNHrXt3XpLpUk8TbA,NkXcAwUXlYoCS8ggzC6rqA,wYgSafshbu0JpKh-LiPisA,1.0,1,0,0,If they ever called me back Id know the level ...,2017-11-06 17:40:11
1,3f4pqNIclMMuTxgIkVQ83g,sh9lnj7mCyL01Hde78O9gA,wYgSafshbu0JpKh-LiPisA,5.0,1,1,0,Rich and his team of skilled electricians do f...,2011-03-21 16:10:52
2,K7Z3yZU9uW5-t6moEyRaig,3SgYeDm2aIf6RkWzTGsNCg,xQHoR3VNqBZODz30FwahnA,5.0,0,0,1,Our 2.5 year old lab/hound has been going to H...,2018-01-10 04:36:45
3,U-wrXnz3J2_2TGeTCEHIqw,d5WLqmTMvmL7-RmUDVKqqQ,xQHoR3VNqBZODz30FwahnA,5.0,4,4,0,We LOVE the Hound Huddle. Our dogs always com...,2014-05-28 21:17:35
4,4OmqSql5ETH1vkR15IBnQg,ESbzOOA8LkipP4FT1iaXbw,MIk9D2NemI-F4AeopYrTzw,5.0,2,0,0,This is my second review of Country View Vet. ...,2017-03-29 18:21:11
...,...,...,...,...,...,...,...,...,...
853,AiUwBtmlbXPQB7DcayiK_A,rubuMDJC7K-5SH4ST2ygfg,Jdtyt2bHpo14dQX5SF_hXQ,4.0,0,0,0,Food was great. The bartender was great. Our w...,2014-08-24 14:54:14
854,JJrA3LNNUqcQdsypONx19A,B2bv64N4vXoWjhlpGYFzbw,Syy4Gr1JrxICbiDm64go3Q,2.0,6,10,3,We were excited to try the new location in Ore...,2015-07-27 07:04:39
855,loozPuBZquuh9F-Ryu4jxg,SdJE4TJ1R8rbA3XfFhUDqg,Syy4Gr1JrxICbiDm64go3Q,2.0,5,10,2,"I really wanted to like this, but I'm continua...",2015-09-23 19:37:45
856,xNaOeLPLTZsyvM_Bv-_Urw,xHzZOs8vw7XczppeVWVigQ,pr9GzyrtkWLt67sKYOhXYg,5.0,3,0,0,"Haven't had our dogs here for nearly a year, s...",2013-06-12 18:09:56


In [62]:
def pivot_ratings(df):
    return df.pivot_table(values='stars', columns='user_id', index='business_id')

In [63]:
pivot_ratings(reviews)

user_id,-03l1FfZrjW5VOr8MT6Yvg,-FgQh77sRF5NGagvCM4ONg,-JRAgeQHfHK20vDTmks0CQ,09QcQYf_t1NEq_AkjJuABA,0FcoFBVkFwZTkWwj3gpUsA,0OdEK86K-esAQ-5k8V9DKA,0WBrn2PHBlYkgvOGr1Yj_Q,0cM-p002vkEVphHv5aTLwA,0seY72QrAbvP4Nh946q9aw,0sk5SochZe8mfn7ACXv8HA,...,yUcKI85MTo3jW4vao67eQw,y_66fMS0Zp6t1hmWKX6JOQ,yf8CKJxMLJFi5R9pXkixuw,yhg2Fgg1M5pnHAbWGdc0Bg,z1RhighiPWszc0n6VWOjXQ,z497rLw1L767W-9Umwgt1g,z4Y8ZWUtQHHThfnCCP4XEA,zbAvU5fpdI4zBEL9Ul30Lw,zgkL5o9Crf0AtHLCrkpkGA,zmr4gUzWiaPeEhIfeKXOvA
business_id,,,,,,,,,,,,,,,,,,,,,
-GrvhyzeZxEQKCtwPndpOw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-y_ImQUTt8NNzqPdKzejag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0JOuIE9U2NJ34Plkrx2k-Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0pTlxo-g1bM6cqonMcDTeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0t09hVqwr2wViq4xc6UGnQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yVdY0hheTMN74MI53uf9cw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yWM4_zhjrRgvSK9YYyvl0w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
ymAvuLRE_B1wVs6eXrC7Iw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
